In [1]:
import numpy as np
import xarray as xr
import os
import glob
from pathlib import Path 
import pathlib
from pprint import pprint
import matplotlib.pyplot as plt
import time
import requests
import s3fs
from os.path import dirname, join
import datetime
import myDate
import netCDF4

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

In [2]:
# alongtrack_file_dir = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_nosicapplied/')
# alongtrack_file_dir_randomnoise = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/')
# alongtrack_file_dir_missingdata = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_missingdata_nosicapplied/')
# alongtrack_file_dir_orbiterror_2cm = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/')
# alongtrack_file_dir_3errors_2cm = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/')
grids_file_dir = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/')
grids_file_dir_randomnoise = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/')
grids_file_dir_missingdata = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/')
grids_file_dir_orbiterror_2cm = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/')
grids_file_dir_3errors_2cm = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/')
grids_file_dir_beckley = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/')

main_dir = Path('/home/jpluser/ECCO_GMSL/')
output_dir = Path('/home/jpluser/ECCO_GMSL/Data/GMSL/')
fig_dir = Path('/home/jpluser/ECCO_GMSL/Figures/')

# Compute GMSL from simple gridder of Beckley along track data daily files

In [3]:
beckley_grids_files = list(grids_file_dir_beckley.glob('ssha_global_half_deg_*nc'))
beckley_grids_files=np.sort(beckley_grids_files)
print(beckley_grids_files[0:5])
print('...')
print(beckley_grids_files[-5:-1])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19920927.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921007.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921017.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921027.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921106.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171115.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171125.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171205.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171215.n

In [6]:
grids = xr.open_mfdataset(
        paths=beckley_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )  

<xarray.Dataset> Size: 3GB
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...
    source:           Combination of MERGED_ALT along track instruments

In [9]:
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset> Size: 2GB
Dimensions:    (longitude: 720, latitude: 264, time: 923)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...
    source:           Combination of MERGED_ALT along track instruments

In [6]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))

In [7]:
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
weights.shape

(923, 264, 720)

In [8]:
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [9]:
# make new DataArray objects
gmsl_grids_beckley = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids_beckley = gmsl_grids_beckley.assign_coords({'time':grids.time.data})
gmsl_grids_beckley.name = 'gmsl_grids_beckley'
gmsl_grids_beckley

<xarray.DataArray 'gmsl_grids_beckley' (time: 923)> Size: 4kB
array([ 1.69078968e-02,  1.22925006e-02,  5.12305181e-03, -2.19792710e-03,
       -4.06668615e-03, -6.64573582e-03, -2.11527734e-03, -1.14462320e-02,
       -1.45043293e-02, -1.74436271e-02, -1.88453346e-02, -2.26884615e-02,
       -2.23314259e-02, -2.40905453e-02, -2.02088933e-02, -1.89829972e-02,
       -1.99991670e-02, -1.91079061e-02, -1.94311906e-02, -2.31938921e-02,
       -2.08824035e-02, -1.86882280e-02, -2.18924358e-02, -2.16961987e-02,
       -2.36001033e-02, -2.31348369e-02, -2.14335956e-02, -2.03797575e-02,
       -1.93761177e-02, -1.89929288e-02, -1.43261943e-02, -1.81000289e-02,
       -1.63723864e-02, -1.35198329e-02, -1.31252529e-02, -1.28582120e-02,
       -1.41602689e-02, -9.68575291e-03, -6.68572308e-03, -8.80166516e-03,
       -1.29868602e-02, -1.21212536e-02, -1.35079222e-02, -1.48071954e-02,
       -1.17592970e-02, -9.72176902e-03, -1.14215296e-02, -1.41173787e-02,
       -1.05190007e-02, -1.19873947e-02, -1.90570801e-02, -2.01623477e-02,
       -1.82843711e-02, -1.60147399e-02, -1.48031795e-02, -1.39125763e-02,
       -1.45940073e-02, -1.59441214e-02, -1.74911264e-02, -1.89811327e-02,
       -1.92872398e-02, -1.95422471e-02, -1.88861806e-02, -1.84019376e-02,
       -2.42679678e-02, -1.77722406e-02, -1.56562086e-02, -1.65406577e-02,
       -1.39443502e-02, -1.33155202e-02, -1.09935226e-02, -1.00437356e-02,
       -7.76944822e-03, -8.11815914e-03, -5.58401877e-03, -5.82845928e-03,
       -4.42570262e-03, -2.48672115e-03, -5.35771344e-03, -9.86848027e-03,
...
        6.17260821e-02,  5.88918738e-02,  6.22275062e-02,  6.12512603e-02,
        5.88533916e-02,  5.69612235e-02,  5.29930666e-02,  5.76765947e-02,
        5.95689416e-02,  5.55902310e-02,  5.77683114e-02,  5.58906086e-02,
        5.58774211e-02,  5.30412532e-02,  5.64005896e-02,  5.10226078e-02,
        5.19252084e-02,  5.03525101e-02,  4.94241379e-02,  4.87880781e-02,
        4.76776771e-02,  5.11273891e-02,  5.19718938e-02,  5.30977473e-02,
        5.29277176e-02,  5.31908311e-02,  5.14503159e-02,  5.21013886e-02,
        5.23775406e-02,  5.35147823e-02,  5.97303249e-02,  6.17798716e-02,
        6.10412098e-02,  5.81164286e-02,  6.01729043e-02,  6.31641522e-02,
        6.32677749e-02,  6.41530827e-02,  6.50055856e-02,  6.46272078e-02,
        6.00098744e-02,  5.96902855e-02,  5.74641638e-02,  5.69302924e-02,
        5.59988245e-02,  5.36370538e-02,  5.10248765e-02,  5.27026169e-02,
        5.25101796e-02,  5.31482399e-02,  5.41420504e-02,  5.27642220e-02,
        5.18646277e-02,  5.22756465e-02,  5.01393415e-02,  4.87993285e-02,
        4.85423058e-02,  4.89996560e-02,  5.02156578e-02,  5.23073487e-02,
        5.20371869e-02,  5.28719090e-02,  5.16298823e-02,  5.63817807e-02,
        5.80990314e-02,  5.99170849e-02,  6.02247044e-02,  6.28706142e-02,
        6.41483217e-02,  6.29180223e-02,  6.42790198e-02,  6.48969039e-02,
        6.75930232e-02,  6.57676682e-02,  6.73445165e-02,  6.72886148e-02,
        6.67184889e-02,  6.61361068e-02,  6.26157001e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

In [10]:
fname = output_dir / ('GMSL_Beckley_dailyfiles_grids.nc')
gmsl_grids_beckley.to_netcdf(fname)

# Compute GMSL from ECCO synthetic grids with simple weighing

### No error

In [11]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-25.nc')]


In [12]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [13]:
grids=grids.where(np.array(grids.counts)>=100)

In [14]:
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 924, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [15]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [16]:
# make new DataArray objects
gmsl_grids = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids = gmsl_grids.assign_coords({'time':grids.time})
gmsl_grids.name = 'gmsl_grids'
gmsl_grids

<xarray.DataArray 'gmsl_grids' (time: 924)> Size: 4kB
array([-3.35424878e-02, -3.15704942e-02, -2.97211539e-02, -3.01636010e-02,
       -3.12151089e-02, -3.31653394e-02, -3.13523225e-02, -3.28133032e-02,
       -3.32171395e-02, -3.89642678e-02, -4.09371518e-02, -4.06149812e-02,
       -4.10725400e-02, -4.25708964e-02, -4.29418050e-02, -4.00319248e-02,
       -3.97198014e-02, -3.81834544e-02, -4.08023484e-02, -3.87738608e-02,
       -3.80125158e-02, -3.72792259e-02, -3.85224000e-02, -3.85570079e-02,
       -3.71993519e-02, -3.69450487e-02, -3.79394032e-02, -3.74104530e-02,
       -3.90305296e-02, -3.77354510e-02, -3.65432948e-02, -3.73000205e-02,
       -3.46635245e-02, -3.37467901e-02, -3.22100073e-02, -3.20308581e-02,
       -3.04958392e-02, -2.84147058e-02, -2.76431981e-02, -2.76976041e-02,
       -2.94093434e-02, -3.01839449e-02, -3.28236297e-02, -3.20038199e-02,
       -2.84981038e-02, -2.69383211e-02, -3.16333100e-02, -3.23781297e-02,
       -3.04484572e-02, -3.41204852e-02, -3.82368490e-02, -3.93310115e-02,
       -3.51031087e-02, -3.32544148e-02, -3.21704932e-02, -3.33084464e-02,
       -3.17383409e-02, -3.33540998e-02, -3.32428813e-02, -3.63176242e-02,
       -3.59278508e-02, -3.55076827e-02, -3.85951065e-02, -3.94888073e-02,
       -4.01364155e-02, -3.90678607e-02, -3.73505242e-02, -3.68583165e-02,
       -3.56557406e-02, -3.55226286e-02, -3.19277383e-02, -3.21692564e-02,
       -2.96279062e-02, -2.96319500e-02, -2.67997812e-02, -2.76168641e-02,
       -2.54191849e-02, -2.63111517e-02, -2.77558900e-02, -2.83813346e-02,
...
        4.37458865e-02,  3.97428311e-02,  3.82923037e-02,  4.21097539e-02,
        4.14056778e-02,  3.79714593e-02,  3.84165794e-02,  3.66020799e-02,
        3.84920910e-02,  3.60000916e-02,  3.88026536e-02,  3.66407149e-02,
        3.77580263e-02,  3.71986590e-02,  3.92618664e-02,  3.86012867e-02,
        3.66206430e-02,  3.80331464e-02,  3.72664817e-02,  3.69350947e-02,
        3.71422283e-02,  3.65053974e-02,  3.54013294e-02,  3.87486592e-02,
        3.91535535e-02,  3.99635024e-02,  4.18622866e-02,  4.32719141e-02,
        4.24299762e-02,  4.22969982e-02,  4.37262245e-02,  4.64143194e-02,
        4.61615846e-02,  4.81634997e-02,  4.73340675e-02,  4.88049723e-02,
        4.68526110e-02,  4.66375165e-02,  4.47805263e-02,  4.50398512e-02,
        4.24686484e-02,  3.87021080e-02,  3.68108116e-02,  3.75178941e-02,
        3.73386666e-02,  3.78090888e-02,  3.77154872e-02,  3.77303623e-02,
        3.93507890e-02,  4.03331146e-02,  4.11640443e-02,  4.17089835e-02,
        3.98945399e-02,  3.98719832e-02,  4.25944999e-02,  4.12794501e-02,
        4.17962968e-02,  4.03562300e-02,  4.10734154e-02,  4.33004610e-02,
        4.41990681e-02,  4.67037037e-02,  4.71757129e-02,  4.78542522e-02,
        4.96560708e-02,  4.73886207e-02,  4.85190451e-02,  4.91735190e-02,
        5.22068068e-02,  5.36501966e-02,  5.42327762e-02,  5.65823540e-02,
        5.48892319e-02,  5.58058135e-02,  5.36121055e-02,  5.50906882e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04

### Random noise

In [17]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_randomnoise.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-25.n

In [18]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [19]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 924, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [20]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [21]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [22]:
# ssh_weighted_mean_randomnoise = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_randomnoise.SSHA

In [23]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_randomnoise=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [24]:
# make new DataArray objects
gmsl_grids_randomnoise = xr.DataArray(ssh_weighted_mean_randomnoise, dims=['time'])
gmsl_grids_randomnoise = gmsl_grids_randomnoise.assign_coords({'time':grids.time})
gmsl_grids_randomnoise.name = 'gmsl_grids'
gmsl_grids_randomnoise

<xarray.DataArray 'gmsl_grids' (time: 924)> Size: 4kB
array([-3.35439928e-02, -3.14961411e-02, -2.96955295e-02, -3.01715676e-02,
       -3.12212426e-02, -3.32428589e-02, -3.13365236e-02, -3.27144042e-02,
       -3.32195126e-02, -3.89343612e-02, -4.09728475e-02, -4.06344309e-02,
       -4.10698019e-02, -4.26040664e-02, -4.29627709e-02, -4.00259085e-02,
       -3.96956764e-02, -3.81502211e-02, -4.08235155e-02, -3.86790819e-02,
       -3.80035564e-02, -3.73036228e-02, -3.85171250e-02, -3.85889038e-02,
       -3.72238569e-02, -3.69588323e-02, -3.79339345e-02, -3.73608842e-02,
       -3.90405767e-02, -3.77635323e-02, -3.64921130e-02, -3.72030027e-02,
       -3.46458070e-02, -3.37154679e-02, -3.22288461e-02, -3.20088863e-02,
       -3.05550434e-02, -2.84575466e-02, -2.76296698e-02, -2.76878159e-02,
       -2.94627231e-02, -3.01479753e-02, -3.28258201e-02, -3.19891274e-02,
       -2.84210797e-02, -2.68859062e-02, -3.15651707e-02, -3.24446186e-02,
       -3.04676238e-02, -3.40826437e-02, -3.82338949e-02, -3.92751694e-02,
       -3.50699164e-02, -3.32494713e-02, -3.21567543e-02, -3.33886519e-02,
       -3.17440033e-02, -3.34158614e-02, -3.32777835e-02, -3.62868048e-02,
       -3.58817317e-02, -3.55755910e-02, -3.86668593e-02, -3.94885354e-02,
       -4.01889384e-02, -3.90219241e-02, -3.72957140e-02, -3.68025042e-02,
       -3.55644338e-02, -3.55160497e-02, -3.19086611e-02, -3.21511626e-02,
       -2.96035875e-02, -2.96478812e-02, -2.67417319e-02, -2.76076477e-02,
       -2.53475774e-02, -2.64268350e-02, -2.77953539e-02, -2.84292400e-02,
...
        4.38148044e-02,  3.97625156e-02,  3.83619331e-02,  4.20923829e-02,
        4.13834117e-02,  3.80008668e-02,  3.83944698e-02,  3.65953930e-02,
        3.84437665e-02,  3.60391289e-02,  3.88184376e-02,  3.66135128e-02,
        3.78252640e-02,  3.72550897e-02,  3.92846763e-02,  3.85551751e-02,
        3.66374664e-02,  3.80322151e-02,  3.72580998e-02,  3.69867571e-02,
        3.72011028e-02,  3.64696421e-02,  3.54155749e-02,  3.87519524e-02,
        3.91711704e-02,  3.99388000e-02,  4.18608598e-02,  4.32350151e-02,
        4.24052998e-02,  4.22171094e-02,  4.37053740e-02,  4.64437120e-02,
        4.61896099e-02,  4.82527725e-02,  4.74214181e-02,  4.87932861e-02,
        4.68996204e-02,  4.65480871e-02,  4.48692106e-02,  4.50842232e-02,
        4.24565785e-02,  3.87566164e-02,  3.69025990e-02,  3.75227630e-02,
        3.73392478e-02,  3.78882922e-02,  3.77085544e-02,  3.77608836e-02,
        3.93036380e-02,  4.03353088e-02,  4.11775932e-02,  4.16646414e-02,
        3.99061143e-02,  3.98774110e-02,  4.25800793e-02,  4.12698016e-02,
        4.18523289e-02,  4.03320715e-02,  4.10673991e-02,  4.32931110e-02,
        4.42415364e-02,  4.67092730e-02,  4.72295545e-02,  4.78150770e-02,
        4.96581122e-02,  4.73969840e-02,  4.84293476e-02,  4.92343456e-02,
        5.21924421e-02,  5.36100417e-02,  5.42309545e-02,  5.65788746e-02,
        5.48628494e-02,  5.58427460e-02,  5.36310561e-02,  5.49648963e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04

### Missing data

In [25]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_missingdata.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-25.n

In [26]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [27]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [28]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [29]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [30]:
# ssh_weighted_mean_missingdata = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_missingdata.SSHA

In [31]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_missingdata=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [32]:
# make new DataArray objects
gmsl_grids_missingdata = xr.DataArray(ssh_weighted_mean_missingdata, dims=['time'])
gmsl_grids_missingdata = gmsl_grids_missingdata.assign_coords({'time':grids.time})
gmsl_grids_missingdata.name = 'gmsl_grids'
gmsl_grids_missingdata

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 4kB
array([-3.36866379e-02, -3.16328220e-02, -3.02344114e-02, -3.02768350e-02,
       -3.13681066e-02, -3.35930064e-02, -3.17269154e-02, -3.30222026e-02,
       -3.27981673e-02, -3.87889631e-02, -4.05341722e-02, -4.02434617e-02,
       -4.07427289e-02, -4.20269370e-02, -4.27076183e-02, -3.99394110e-02,
       -3.95827815e-02, -3.77041437e-02, -4.06352244e-02, -3.84921432e-02,
       -3.79117504e-02, -3.71440351e-02, -3.82645056e-02, -3.85763384e-02,
       -3.72440293e-02, -3.67893092e-02, -3.79876830e-02, -3.72524895e-02,
       -3.91771980e-02, -3.76212485e-02, -3.54992598e-02, -3.73194590e-02,
       -3.46337184e-02, -3.34925875e-02, -3.21862064e-02, -3.20242420e-02,
       -3.03880889e-02, -2.83802785e-02, -2.73909140e-02, -2.76632980e-02,
       -2.95344815e-02, -3.02564558e-02, -3.32846604e-02, -3.16983722e-02,
       -2.85323393e-02, -2.67223008e-02, -3.13205458e-02, -3.22974361e-02,
       -2.99982522e-02, -3.36991213e-02, -3.81467491e-02, -3.94601747e-02,
       -3.48746814e-02, -3.27942111e-02, -3.20911482e-02, -3.29896435e-02,
       -3.14702094e-02, -3.32737416e-02, -3.29849608e-02, -3.63111049e-02,
       -3.58754434e-02, -3.54589447e-02, -3.84269841e-02, -3.95816267e-02,
       -4.05076779e-02, -3.90828550e-02, -3.74893509e-02, -3.66925709e-02,
       -3.56261581e-02, -3.56963687e-02, -3.21107283e-02, -3.19834761e-02,
       -2.93035991e-02, -3.00774481e-02, -2.66945381e-02, -2.75477041e-02,
       -2.55920552e-02, -2.62307543e-02, -2.85877753e-02, -2.84998585e-02,
...
        4.62957546e-02,  4.75640707e-02,  4.48645465e-02,  4.34629917e-02,
        3.94962169e-02,  3.80720049e-02,  4.21115048e-02,  4.10847366e-02,
        3.77914235e-02,  3.83952968e-02,  3.65739018e-02,  3.88033353e-02,
        3.60120349e-02,  3.89458276e-02,  3.65566500e-02,  3.79025787e-02,
        3.73351350e-02,  3.94119546e-02,  3.85618471e-02,  3.67955491e-02,
        3.80028188e-02,  3.72958072e-02,  3.69950607e-02,  3.71581614e-02,
        3.65905575e-02,  3.54670174e-02,  3.87137309e-02,  3.92026044e-02,
        4.00390513e-02,  4.20418791e-02,  4.36415970e-02,  4.23786081e-02,
        4.23890837e-02,  4.38673906e-02,  4.65786196e-02,  4.63752300e-02,
        4.80895862e-02,  4.73351628e-02,  4.87441942e-02,  4.66747880e-02,
        4.64442149e-02,  4.46666777e-02,  4.48983982e-02,  4.21386547e-02,
        3.86426114e-02,  3.67927104e-02,  3.77861969e-02,  3.73428650e-02,
        3.79213728e-02,  3.79495621e-02,  3.78687158e-02,  3.93107124e-02,
        4.04561684e-02,  4.14000265e-02,  4.16581109e-02,  3.98999825e-02,
        3.98148298e-02,  4.25611138e-02,  4.12010513e-02,  4.15342860e-02,
        4.02770378e-02,  4.10949141e-02,  4.32692915e-02,  4.41222116e-02,
        4.66942452e-02,  4.70420383e-02,  4.78725806e-02,  4.98074479e-02,
        4.73555848e-02,  4.85071056e-02,  4.92470600e-02,  5.23025356e-02,
        5.37153184e-02,  5.40612191e-02,  5.64643331e-02,  5.47644719e-02,
        5.57320826e-02,  5.34459949e-02,  5.42023964e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04

### Orbit Error - 2cm

In [33]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_orbiterror_2cm.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplie

In [34]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (longitude: 720, latitude: 360, time: 924)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 958MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [35]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 924, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 703MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [36]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_orbiterror=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [37]:
# make new DataArray objects
gmsl_grids_orbiterror_2cm = xr.DataArray(ssh_weighted_mean_orbiterror, dims=['time'])
gmsl_grids_orbiterror_2cm = gmsl_grids_orbiterror_2cm.assign_coords({'time':grids.time})
gmsl_grids_orbiterror_2cm.name = 'gmsl_grids'
gmsl_grids_orbiterror_2cm

<xarray.DataArray 'gmsl_grids' (time: 924)> Size: 4kB
array([-3.35367732e-02, -3.21325958e-02, -2.95429155e-02, -3.06006595e-02,
       -3.14139910e-02, -3.30315568e-02, -3.17447670e-02, -3.31899524e-02,
       -3.28539759e-02, -3.90048698e-02, -4.02611755e-02, -4.05071080e-02,
       -4.10477258e-02, -4.24200296e-02, -4.33974154e-02, -4.01309840e-02,
       -3.98296155e-02, -3.80239412e-02, -4.07181308e-02, -3.84771377e-02,
       -3.85718159e-02, -3.75445560e-02, -3.83779407e-02, -3.80385853e-02,
       -3.73588540e-02, -3.67544442e-02, -3.74791361e-02, -3.66970450e-02,
       -3.90611216e-02, -3.76413614e-02, -3.64376232e-02, -3.72899920e-02,
       -3.46074328e-02, -3.34267952e-02, -3.22759971e-02, -3.20088491e-02,
       -3.01253870e-02, -2.90537160e-02, -2.75146365e-02, -2.80038454e-02,
       -2.89831068e-02, -3.04747801e-02, -3.33081186e-02, -3.13842334e-02,
       -2.81910338e-02, -2.72285305e-02, -3.14684324e-02, -3.18313800e-02,
       -3.01479511e-02, -3.42839360e-02, -3.81004252e-02, -3.91566791e-02,
       -3.54191139e-02, -3.29465941e-02, -3.21340635e-02, -3.33554596e-02,
       -3.22607346e-02, -3.34330648e-02, -3.32352482e-02, -3.61278169e-02,
       -3.60889621e-02, -3.53131220e-02, -3.84961478e-02, -3.91454808e-02,
       -4.01996411e-02, -3.87378335e-02, -3.69711779e-02, -3.72086763e-02,
       -3.55418101e-02, -3.56511287e-02, -3.21242474e-02, -3.15444134e-02,
       -2.94190776e-02, -3.00582983e-02, -2.72468422e-02, -2.75723469e-02,
       -2.54666731e-02, -2.65488308e-02, -2.78902575e-02, -2.82412115e-02,
...
        4.35181968e-02,  4.03263159e-02,  3.81550975e-02,  4.23383974e-02,
        4.19135131e-02,  3.81355621e-02,  3.86449844e-02,  3.69299799e-02,
        3.81816924e-02,  3.54492441e-02,  3.91576998e-02,  3.69018577e-02,
        3.72899510e-02,  3.67284045e-02,  3.88685055e-02,  3.84703279e-02,
        3.64951007e-02,  3.82696018e-02,  3.74251120e-02,  3.69286388e-02,
        3.80502604e-02,  3.64125632e-02,  3.56607884e-02,  3.93664315e-02,
        3.91520225e-02,  4.06179801e-02,  4.21304479e-02,  4.32967842e-02,
        4.26656641e-02,  4.26983535e-02,  4.36857529e-02,  4.60858122e-02,
        4.56900708e-02,  4.80811335e-02,  4.78889085e-02,  4.89505827e-02,
        4.67896387e-02,  4.74779010e-02,  4.49194424e-02,  4.52404283e-02,
        4.24517654e-02,  3.85656804e-02,  3.68966945e-02,  3.67054902e-02,
        3.78208235e-02,  3.77542190e-02,  3.76968421e-02,  3.73386741e-02,
        3.93109843e-02,  4.00038548e-02,  4.13612388e-02,  4.14314680e-02,
        4.06732932e-02,  3.98184992e-02,  4.25274596e-02,  4.14544940e-02,
        4.17275839e-02,  4.05249856e-02,  4.12241109e-02,  4.28448804e-02,
        4.41859663e-02,  4.60649207e-02,  4.69852909e-02,  4.84898798e-02,
        4.91553806e-02,  4.75201756e-02,  4.76363115e-02,  4.95460629e-02,
        5.21366522e-02,  5.39137907e-02,  5.44627197e-02,  5.66628166e-02,
        5.52364811e-02,  5.58423363e-02,  5.32598980e-02,  5.35157621e-02],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04

### 3 errors: random noise + missing data + orbit error - 2cm

In [38]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_3errors_2cm.glob('*SSHA_gridded*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:-1])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-25.n

In [39]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (longitude: 720, latitude: 360, time: 923)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 957MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [40]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66))
grids

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * longitude  (longitude) float32 3kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * latitude   (latitude) float32 1kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    mask       (time, latitude, longitude) float32 702MB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
Attributes:
    gridding_method:  Gridded using pyresample resample_gauss with roi=600000...

In [41]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_3errors=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [42]:
# make new DataArray objects
gmsl_grids_3errors_2cm = xr.DataArray(ssh_weighted_mean_3errors, dims=['time'])
gmsl_grids_3errors_2cm = gmsl_grids_3errors_2cm.assign_coords({'time':grids.time})
gmsl_grids_3errors_2cm.name = 'gmsl_grids'
gmsl_grids_3errors_2cm

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 4kB
array([-3.36070135e-02, -3.21449786e-02, -3.07996925e-02, -3.02446950e-02,
       -3.18286717e-02, -3.39521356e-02, -3.23365889e-02, -3.30302380e-02,
       -3.23903374e-02, -3.87053005e-02, -4.01303396e-02, -3.98967527e-02,
       -4.11448628e-02, -4.16696109e-02, -4.26705703e-02, -4.01602276e-02,
       -3.97948064e-02, -3.73996496e-02, -4.10879217e-02, -3.81408110e-02,
       -3.80208232e-02, -3.70838307e-02, -3.88043635e-02, -3.94192375e-02,
       -3.76064219e-02, -3.64667363e-02, -3.80177088e-02, -3.73602845e-02,
       -3.94128785e-02, -3.74587625e-02, -3.56062986e-02, -3.70020159e-02,
       -3.42087857e-02, -3.35811973e-02, -3.19821313e-02, -3.19992974e-02,
       -2.94781327e-02, -2.79135741e-02, -2.74956059e-02, -2.80436147e-02,
       -3.01998369e-02, -3.03478371e-02, -3.31766382e-02, -3.12807411e-02,
       -2.80294418e-02, -2.66139079e-02, -3.13676260e-02, -3.19371596e-02,
       -3.00406571e-02, -3.34827863e-02, -3.80449407e-02, -3.92863788e-02,
       -3.53933834e-02, -3.26046385e-02, -3.20729613e-02, -3.33183184e-02,
       -3.16936821e-02, -3.35110314e-02, -3.29957530e-02, -3.66674736e-02,
       -3.59757207e-02, -3.50874141e-02, -3.82094644e-02, -3.94018479e-02,
       -4.06510942e-02, -3.85289155e-02, -3.79966013e-02, -3.71867195e-02,
       -3.56602222e-02, -3.63434851e-02, -3.14267017e-02, -3.16110626e-02,
       -2.96672285e-02, -2.96380017e-02, -2.68783346e-02, -2.79460289e-02,
       -2.57096738e-02, -2.59203091e-02, -2.86005959e-02, -2.80758683e-02,
...
        4.60293591e-02,  4.74453121e-02,  4.50457782e-02,  4.31408584e-02,
        3.98112461e-02,  3.82618159e-02,  4.18093242e-02,  4.03266326e-02,
        3.80988829e-02,  3.88204530e-02,  3.68030332e-02,  3.82745527e-02,
        3.58362794e-02,  3.87860425e-02,  3.64290737e-02,  3.77523713e-02,
        3.71054634e-02,  3.99021842e-02,  3.82639952e-02,  3.68536003e-02,
        3.81825641e-02,  3.69414911e-02,  3.64861302e-02,  3.68988216e-02,
        3.65785770e-02,  3.54795679e-02,  3.92101333e-02,  3.91849540e-02,
        3.99926640e-02,  4.18678820e-02,  4.33557481e-02,  4.29199673e-02,
        4.30766232e-02,  4.43472825e-02,  4.62528616e-02,  4.61196341e-02,
        4.77117896e-02,  4.69891019e-02,  4.85846475e-02,  4.70677689e-02,
        4.62621786e-02,  4.43550646e-02,  4.54731360e-02,  4.18113247e-02,
        3.90074998e-02,  3.63908224e-02,  3.82306091e-02,  3.77439223e-02,
        3.83238196e-02,  3.75210084e-02,  3.76672037e-02,  3.95011157e-02,
        4.07764986e-02,  4.14141305e-02,  4.11783308e-02,  3.98415811e-02,
        3.99121642e-02,  4.25126292e-02,  4.11319137e-02,  4.16917577e-02,
        4.00905050e-02,  4.06988226e-02,  4.29227538e-02,  4.41531949e-02,
        4.67871651e-02,  4.71392088e-02,  4.77613285e-02,  4.97596227e-02,
        4.69374582e-02,  4.80626561e-02,  4.96777520e-02,  5.17809615e-02,
        5.44144027e-02,  5.45064360e-02,  5.61890192e-02,  5.52172363e-02,
        5.56340776e-02,  5.31741679e-02,  5.32641783e-02], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04

### Save to Disk

In [43]:
fname = output_dir / ('GMSL_grids_19922017_noerror_100points_nosicapplied.nc')
gmsl_grids.to_netcdf(fname)

In [44]:
fname = output_dir / ('GMSL_grids_19922017_randomnoise_100points_nosicapplied.nc')
gmsl_grids_randomnoise.to_netcdf(fname)

In [45]:
fname = output_dir / ('GMSL_grids_19922017_missingdata_100points_nosicapplied.nc')
gmsl_grids_missingdata.to_netcdf(fname)

In [46]:
fname = output_dir / ('GMSL_grids_19922017_orbiterror_2cm_100points_nosicapplied.nc')
gmsl_grids_orbiterror_2cm.to_netcdf(fname)

In [47]:
fname = output_dir / ('GMSL_grids_19922017_3errors_2cm_100points_nosicapplied.nc')
gmsl_grids_3errors_2cm.to_netcdf(fname)